In [11]:
import pandas as pd
import ipaddress
import numpy as np
import pandasql as ps

In [12]:
filename = '~/Downloads/log20030214/log20030214.csv'

In [13]:
def pipeline(filename):
    df = pd.read_csv(filename, nrows=20000)
    return df[['ip', 'date', 'time','cik', 'accession']].drop_duplicates()

def to_ipaddress(arr): 
    arr = [np.char.replace(ip, ip[-3:],'000') for ip in arr]
    return [int(ipaddress.IPv4Address(x)) for x in arr]

In [14]:
df = pipeline(filename)

In [15]:
df['ip']= to_ipaddress(df['ip'])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18981 entries, 0 to 19999
Data columns (total 5 columns):
ip           18981 non-null int64
date         18981 non-null object
time         18981 non-null object
cik          18981 non-null float64
accession    18981 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 889.7+ KB


In [16]:
df_geoips = pd.read_csv('~/Downloads/IP2LOCATION-LITE-DB1.CSV')
df_geoips.iloc[:,0] = df_geoips.iloc[:,0].apply(lambda x: int(ipaddress.IPv4Address(x)))
df_geoips.iloc[:,1] = df_geoips.iloc[:,1].apply(lambda x: int(ipaddress.IPv4Address(x)))
df_geoips.columns = ["from_ip","to_ip","code","country"]
sqlcode = '''
    SELECT * 
    FROM df, df_geoips
    WHERE ip BETWEEN from_ip AND to_ip
'''
newdf = ps.sqldf(sqlcode, locals()).drop(columns=['from_ip','to_ip','code'])

In [17]:
newdf['country'].value_counts()

United States of America                                17658
United Kingdom of Great Britain and Northern Ireland      378
Canada                                                    371
Germany                                                    94
Switzerland                                                72
France                                                     64
India                                                      51
Mexico                                                     36
Hong Kong                                                  32
Netherlands                                                23
Antigua and Barbuda                                        22
Spain                                                      16
Ireland                                                    15
Italy                                                      15
Peru                                                       15
Brazil                                                     14
Australi

United States of America                                9416
United Kingdom of Great Britain and Northern Ireland    1142
China                                                    979
Bulgaria                                                 340
Canada                                                   241
Italy                                                    200
Ukraine                                                  182
Netherlands                                              172
Czechia                                                  159
Russian Federation                                       146
India                                                    133
Hong Kong                                                109
Korea (Republic of)                                       93
Australia                                                 89
Spain                                                     82
Greece                                                    50
Brazil                  